<a href="https://colab.research.google.com/github/erobrien6/OBrien_DSPN_S23/blob/main/Exercise9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 9: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [3]:
# WRITE YOUR CODE HERE
# Manually uploaded data to colab due to working directory error
df_lexical <- read.csv("/content/LexicalData.csv")
df_items <- read.csv("/content/Items.csv")

head(df_lexical)
head(df_items)
summary(df_items)
summary(df_lexical)

library(tidyverse)
library(dplyr)

## Remove commas
df_lexical$D_RT <- gsub(',','', df_lexical$D_RT)

## Make numeric
df_lexical$D_RT <- as.numeric(df_lexical$D_RT)

## Join task
df_final <- left_join(df_lexical, df_items, by = c('D_Word' = 'Word'))
# Only include Length and Log_Freq_HAL
df_final <- df_final %>%
  select(c(1:7, 9, 11))

head(df_final)


,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,"1,094",refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<chr>,<int>,<chr>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


  Occurrences        Word               Length         Freq_HAL        
 Min.   :1.000   Length:30959       Min.   : 1.000   Length:30959      
 1st Qu.:1.000   Class :character   1st Qu.: 6.000   Class :character  
 Median :2.000   Mode  :character   Median : 8.000   Mode  :character  
 Mean   :2.022                      Mean   : 7.992                     
 3rd Qu.:3.000                      3rd Qu.:10.000                     
 Max.   :9.000                      Max.   :21.000                     
  Log_Freq_HAL   
 Min.   : 0.000  
 1st Qu.: 4.860  
 Median : 6.332  
 Mean   : 6.315  
 3rd Qu.: 7.856  
 Max.   :16.214  

     Sub_ID          Trial            Type       D_RT          
 Min.   :  1.0   Min.   :  1.0   Min.   :1   Length:62610      
 1st Qu.: 77.0   1st Qu.:124.0   1st Qu.:1   Class :character  
 Median :152.0   Median :251.0   Median :1   Mode  :character  
 Mean   :151.5   Mean   :249.5   Mean   :1                     
 3rd Qu.:227.0   3rd Qu.:375.0   3rd Qu.:1                     
 Max.   :300.0   Max.   :500.0   Max.   :1                     
    D_Word            Outlier             D_Zscore       
 Length:62610       Length:62610       Min.   :-3.03200  
 Class :character   Class :character   1st Qu.:-0.65300  
 Mode  :character   Mode  :character   Median :-0.24000  
                                       Mean   : 0.03338  
                                       3rd Qu.: 0.41600  
                                       Max.   :11.56200  

,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore,Length,Log_Freq_HAL
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>
1,157,1,1,710,browse,false,-0.437,6,8.856
2,67,1,1,1094,refrigerant,false,0.825,11,4.644
3,120,1,1,587,gaining,false,-0.645,7,8.304
4,21,1,1,984,cheerless,false,0.025,9,2.639
5,236,1,1,577,pattered,false,-0.763,8,1.386
6,236,2,1,715,conjures,false,-0.364,8,5.268


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [4]:
# WRITE YOUR CODE HERE
linear_model <- lm(D_RT ~ Length + Log_Freq_HAL + Length*Log_Freq_HAL, data = df_final)
summary(linear_model)



Call:
lm(formula = D_RT ~ Length + Log_Freq_HAL + Length * Log_Freq_HAL, 
    data = df_final)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length:Log_Freq_HAL  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library. 

In [ ]:
# WRITE YOUR CODE HERE
# install.packages('lme4')
library('lme4')


Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [13]:
# WRITE YOUR CODE HERE
mixed_model <- lmer(D_RT ~ + Length + Log_Freq_HAL + Length*Log_Freq_HAL + (1|Sub_ID), data = df_final)
summary(mixed_model)

# Linear Regression T Values
## Length: 29.175
## Log: -3.061
## Interaction: -12.528

# ME T Values
## Length: 36.277
## Log: -4.698
## Interaction: -15.239


Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ +Length + Log_Freq_HAL + Length * Log_Freq_HAL + (1 |  
    Sub_ID)
   Data: df_final

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Length               47.7477     1.3162  36.277
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length:Log_Freq_HAL  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Length L_F_HA
Length      -0.656              
Log_Frq_HAL -0.645  0.917       
Lng:L_F_HAL  0.582 -0.923 -0.942

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> *All of the t-values in both models yield significant effects (p < .05). However, the mixed effects model t-values are all greater in magnitude than the t-values of the linear regression model (for Length, it increased from 29.2 to 36.3, for Log_Freq_HAL it decreased from -3.1 to -4.7, and for the interaction term it decreased from -12.5 to -15.2). This difference suggests that by controlling for subject effects, the p-values become smaller, and we are getting a less conservative model fit on the fixed effects.* 
> 

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [15]:
# WRITE YOUR CODE HERE

ic = AIC(linear_model, mixed_model)
ic
diff(ic$AIC)

,df,AIC
,<dbl>,<dbl>
linear_model,5,914436.4
mixed_model,6,888247.6


[1] -26188.82

> *The mixed effects model appears to be a better fit than the linear model because the AIC is lower, indicating that the ME model can explain more variance in reaction time even when accounting for increased complexity.* 
> 

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> *One example of another random effect that could be controlled for is by trial number. This could be relevant if we were interested in assessing whether particpants experienced task fatigue and performed worse in general after their first trial; conversely, we could control for practice effects and potential improvement over trials.* 
> 

**DUE:** 5pm EST, March 15, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*